In [1]:
#-- Import Required Python Packages
import numpy as np
import pandas as pd

In [2]:
# Load Dataset
data2014 = pd.read_csv('LoanStats3c.csv',header=1)
data2015 = pd.read_csv('LoanStats3d.csv',header=1)

/home/dongwuw/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/dongwuw/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# 1. What is the median loan amount? 
total_loan_amount = pd.concat([data2014['loan_amnt'], data2015['loan_amnt']])
print(total_loan_amount.median())

13750.0


In [4]:
# 2. What fraction of all loans are for the most common purpose? 
purpose = pd.concat([data2014['purpose'], data2015['purpose']])
print(max(purpose.value_counts())/len(purpose.dropna()))

0.598464499546


In [5]:
# 3. Calculate the average interest rate across loans for each purpose. 
# What is the ratio of minimum average rate to the maximum average rate? 
d3 = pd.concat([data2014[['int_rate','purpose']], data2015[['int_rate','purpose']]])
# change percentage to float
d3['int_rate'] = d3['int_rate'].replace('%','',regex=True).astype('float')/100
avg_int_rate_per_purpose = d3.dropna().groupby('purpose')['int_rate'].mean()
print(min(avg_int_rate_per_purpose) / max(avg_int_rate_per_purpose))

0.639979776706


In [6]:
# 4. What is the difference in the fraction of the loans 
# with a 36-month term between 2014 and 2015? 
d4_2014 = np.array(data2014['term'].value_counts())
d4_2015 = np.array(data2015['term'].value_counts())
ratio_2014 = d4_2014[0]/sum(d4_2014)
ratio_2015 = d4_2015[0]/sum(d4_2015)
print(ratio_2014 - ratio_2015)

0.0174723342368


In [7]:
# 5. We will consider all loans that are not in the 'Fully Paid', 'Current', 
# 'In Grace Period' statuses to be in default. Calculate the ratio of 
# the time spent paying the loan, defined as the difference between the 
# last payment date and the issue date, divided by the term of loan. 
# What is the standard deviation of this ratio for all the loans in default? 
d5_old = pd.concat([data2014[['loan_status','issue_d','last_pymnt_d','term']], data2015[['loan_status','issue_d','last_pymnt_d','term']]])
# focus on default status
d5 = d5_old[~d5_old['loan_status'].isin(['Current', 'Fully Paid', 'In Grace Period'])].dropna()
d5['term'] = d5['term'].replace('months','',regex=True).astype('int')
d5['monthdiffratio'] = (pd.to_datetime(d5["last_pymnt_d"]) - pd.to_datetime(d5["issue_d"])).astype('timedelta64[M]') / d5['term']
print(d5['monthdiffratio'].std())

0.19657097306016852


In [8]:
# 6. What is the Pearson correlation coefficient between the 
# total rate of return, as figured from the total payments and 
# the loan amount, and the interest rate? Consider only loans 
# that have reached the end of their term. 
d6_old = pd.concat([data2014[['loan_status','loan_amnt','total_pymnt','int_rate']], data2015[['loan_status','loan_amnt','total_pymnt','int_rate']]])
d6 = d6_old[d6_old['loan_status'].isin(['Fully Paid'])].dropna()
d6['rateofreturn'] = d6['total_pymnt'] / d6['loan_amnt']
d6['int_rate'] = d6['int_rate'].replace('%','',regex=True).astype('float')/100
print(d6['rateofreturn'].corr(d6['int_rate']))

0.54726352362


In [9]:
# 7. Let's find a loan purpose that shows up abnormally often 
# in one state. Call A the probability of a loan going to a 
# specific purpose nationwide. Call B the probability of a 
# loan going to a specific purpose for each state. Out of all 
# (state, purpose) pairs with at least 10 loans, what is the 
# highest ratio of B / A (i.e. the most surprising)? 
d7 = pd.concat([data2014[['addr_state','purpose']], data2015[['addr_state','purpose']]]).dropna()
prob_national = pd.DataFrame(d7['purpose'].value_counts() / len(d7['purpose']) * 100).sort_index()
#print(d7['addr_state'].value_counts()) # state = IA or ID only has one data
d7 = d7[~d7['addr_state'].isin(['ID','IA'])]
g7 = d7.groupby(['addr_state','purpose']).agg({'purpose': 'count'})
g7pct = pd.DataFrame(g7.groupby(level=0).apply(lambda x: 100 * x / float(x.sum())))
g7pct.columns = ['pct']
g7pct.reset_index(inplace=True)
g7pct.columns = ['addr_state','purpose','pct']
res_state = g7pct.pivot(index='purpose', columns='addr_state', values='pct').fillna(0.0)
res_state['national'] = prob_national
#print(res_state)
ratio_state = pd.DataFrame(res_state.div(res_state['national'],axis='index'))
#print(ratio_state)
rs_copy = ratio_state.unstack().copy()
rs_copy.sort_values(inplace=True)
n = 20
temp_res = pd.DataFrame(rs_copy[-n:])
temp_res.reset_index(inplace=True)
temp_res.columns = ['addr_state', 'purpose', 'ratio']
#print(temp_res)
for i in reversed(range(n)):
    if int(g7.loc[temp_res['addr_state'][i],temp_res['purpose'][i]]) > 10:
        print(g7.loc[temp_res['addr_state'][i],temp_res['purpose'][i]])
        print(float(temp_res['ratio'][i]))
        break
#g7.query('addr_state=="DE"')
#g7.loc['DE','educational']

purpose    56
Name: (HI, vacation), dtype: int64
3.2727693086839853


In [10]:
# 8. Group all loans by their sub-grade and calculate their 
# average interest rate and average default rate. Build a 
# linear model to predict the (average) default rate from 
# the (average) interest rate. Find that sub-grade with 
# the largest absolute deviation from this model. What is 
# the deviation of the actual default rate from the predicted value? 
from sklearn import linear_model
# Combine Datasets
d8 = pd.concat([data2014[['sub_grade','int_rate','loan_status']], data2015[['sub_grade','int_rate','loan_status']]]).dropna()
d8['int_rate'] = d8['int_rate'].replace('%','',regex=True).astype('float')/100
# Default = 1, others = 0
d8['loan_status'].replace(['Fully Paid','Current','In Grace Period',\
                           'Late (31-120 days)','Charged Off',\
                           'Late (16-30 days)', 'Default'],\
                          [0,0,0,1,1,1,1],inplace=True)
d8_1 = d8.groupby(['sub_grade']).agg({'int_rate': 'mean'})
d8_2 = d8.groupby(['sub_grade']).agg({'loan_status': 'sum'})
d8_3 = d8.groupby(['sub_grade']).agg({'loan_status': 'count'})
d8_new = pd.concat([d8_1,d8_2/d8_3], axis=1)
d8_new.columns = ['air','adr']#averaged interest rate and averaged default rate
# Fit a linear regression model
x = d8_new.air.values
y = d8_new.adr.values
n = x.shape[0]
x = x.reshape(n,1)
y = y.reshape(n,1)
regr = linear_model.LinearRegression()
regr.fit(x,y)
y_hat = regr.predict(x)
deviation = y-y_hat
print(np.argmax(abs(deviation)))# G5 has the largest absolute deviation from this model
print("%.10f" % deviation[np.argmax(abs(deviation))])

34
0.0304874826
